In [ ]:
# signal_analysis.py

import pandas as pd
import numpy as np
from scipy.signal import butter, filtfilt, welch
from ahrs.filters import Mahony
import streamlit as st # Necesario para st.warning en caso de problemas
from config import FS, VENTANA_DURACION_SEG, SOLAPAMIENTO_VENTANA, TREMOR_FILTER_BAND, FILTER_ORDER, SENSOR_COLS

def filter_tremor_signal(signal):
    """
    Aplica un filtro Butterworth pasabanda a la señal de temblor.
    """
    nyquist = 0.5 * FS
    low_cut = TREMOR_FILTER_BAND[0] / nyquist
    high_cut = TREMOR_FILTER_BAND[1] / nyquist

    # Diseño del filtro
    b, a = butter(N=FILTER_ORDER, Wn=[low_cut, high_cut], btype='band')

    # Aplicación del filtro sin desfase
    return filtfilt(b, a, signal)

def quaternion_to_rotation_matrix(q):
    """
    Convierte un cuaternión a una matriz de rotación 3x3.
    """
    w, x, y, z = q
    return np.array([
        [1 - 2*(y**2 + z**2),          2*(x*y - z*w),          2*(x*z + y*w)],
        [2*(x*y + z*w),          1 - 2*(x**2 + z**2),        2*(y*z - x*w)],
        [2*(x*z - y*w),          2*(y*z + x*w),          1 - 2*(x**2 + y**2)]
    ])

def analyze_tremor_windows(df_sensor_data):
    """
    Realiza el análisis de temblor por ventanas, incluyendo compensación de gravedad
    y cálculo de métricas (frecuencia, RMS, amplitud).
    """
    if df_sensor_data.empty:
        st.warning("DataFrame de entrada vacío para análisis de temblor.")
        return pd.DataFrame(), pd.DataFrame()

    # Asegurarse de que las columnas del sensor realmente existen y tienen datos
    current_sensor_cols = [col for col in SENSOR_COLS if col in df_sensor_data.columns and df_sensor_data[col].notna().any()]

    if not current_sensor_cols:
        st.warning("No se encontraron columnas de sensor válidas para el análisis de temblor.")
        return pd.DataFrame(), pd.DataFrame()

    # Extraer datos de acelerómetro y giroscopio
    # Asegúrate de que las columnas para acc y gyr existen antes de intentar acceder a ellas
    acc_cols = [col for col in ['Acel_X', 'Acel_Y', 'Acel_Z'] if col in current_sensor_cols]
    gyr_cols = [col for col in ['GiroX', 'GiroY', 'GiroZ'] if col in current_sensor_cols]

    if not acc_cols or not gyr_cols:
        st.warning("Datos insuficientes de acelerómetro o giroscopio para el filtro Mahony.")
        return pd.DataFrame(), pd.DataFrame()

    acc = df_sensor_data[acc_cols].to_numpy()
    gyr = np.radians(df_sensor_data[gyr_cols].to_numpy())

    # --- Compensación de Gravedad con Mahony ---
    mahony = Mahony(gyr=gyr, acc=acc, frequency=FS)
    Q = mahony.Q

    linear_accelerations_magnitude = []
    g_world_vector = np.array([0.0, 0.0, 9.81]) # Vector de gravedad en el marco global

    for i in range(len(acc)):
        q = Q[i]
        acc_measured = acc[i]
        R_W_B = quaternion_to_rotation_matrix(q) # Matriz de rotación del mundo al cuerpo

        # Rotar el vector de gravedad del mundo al marco del sensor
        gravity_in_sensor_frame = R_W_B @ g_world_vector

        # Aceleración lineal = Aceleración medida - Aceleración de la gravedad
        linear_acc_sensor_frame = acc_measured - gravity_in_sensor_frame

        linear_accelerations_magnitude.append(np.linalg.norm(linear_acc_sensor_frame))

    movimiento_lineal = np.array(linear_accelerations_magnitude)
    señal_filtrada = filter_tremor_signal(movimiento_lineal) # Usa la función de filtrado

    # --- Análisis por Ventanas ---
    resultados_por_ventana = []

    window_samples = int(FS * VENTANA_DURACION_SEG)
    overlap_samples = int(window_samples * SOLAPAMIENTO_VENTANA)
    step_samples = window_samples - overlap_samples

    if step_samples <= 0 or len(señal_filtrada) < window_samples:
        st.warning("Parámetros de ventana inválidos o datos insuficientes para formar ventanas.")
        return pd.DataFrame(), pd.DataFrame()

    num_windows = (len(señal_filtrada) - window_samples) // step_samples + 1

    for i in range(num_windows):
        start = i * step_samples
        end = start + window_samples
        window = señal_filtrada[start:end]

        if len(window) < window_samples:
            continue # Skip incomplete windows

        # Asegurarse de que el segmento tenga varianza para FFT
        if np.var(window) < 1e-9: # Umbral pequeño para detectar señal casi constante
            freq_dominante = np.nan
            rms = 0.0
            amp_g = 0.0
            amp_cm = 0.0
        else:
            # Eliminar la media para el análisis de frecuencia (señal de CA)
            segmento_dc_removed = window - np.mean(window)

            # Espectro de Potencia (PSD)
            f, Pxx = welch(segmento_dc_removed, fs=FS, nperseg=window_samples, scaling='density')

            # Filtrar solo el rango de temblor (ej. 3-12 Hz) para buscar la frecuencia dominante
            tremor_freq_range_mask = (f >= 3) & (f <= 12)
            f_tremor = f[tremor_freq_range_mask]
            Pxx_tremor = Pxx[tremor_freq_range_mask]

            if len(Pxx_tremor) > 0:
                idx_max_psd = np.argmax(Pxx_tremor)
                freq_dominante = f_tremor[idx_max_psd]
            else:
                freq_dominante = np.nan # No se encontró frecuencia dominante en el rango de temblor

            # Cálculo de RMS y Amplitud
            rms = np.sqrt(np.mean(window**2)) # RMS de la señal original en la ventana
            amp_g = (np.max(window) - np.min(window))/2 # Amplitud pico a pico / 2

            # Conversión de aceleración (g) a desplazamiento (cm)
            if pd.notna(freq_dominante) and freq_dominante > 1.5: # Evitar división por cero o por frecuencias muy bajas
                # Fórmula A = a / (2 * pi * f)^2. Multiplicar por 100 para cm.
                # Se multiplica por 2 porque amp_g es 'pico' y la fórmula suele ser para RMS, o para ajustar
                # entre diferentes definiciones de amplitud. Tu código original lo tenía, se mantiene.
                amp_cm = (amp_g * 100) / ((2 * np.pi * freq_dominante) ** 2) * 2
            else:
                amp_cm = np.nan # No se puede calcular si la frecuencia dominante no es válida o muy baja

        resultados_por_ventana.append({
           'Ventana': i + 1, # Empezar desde 1 para legibilidad
           'Frecuencia Dominante (Hz)': freq_dominante,
           'RMS (m/s2)': rms,
           'Amplitud Temblor (g)': amp_g, # Mantenemos esta para trazabilidad si es útil
           'Amplitud Temblor (cm)': amp_cm
         })

    df_por_ventana = pd.DataFrame(resultados_por_ventana)

    if not df_por_ventana.empty:
        # Excluir 'Amplitud Temblor (g)' de este promedio si no se usa para la predicción
        promedio = df_por_ventana[['Frecuencia Dominante (Hz)', 'RMS (m/s2)', 'Amplitud Temblor (cm)']].mean(numeric_only=True).to_dict()
        df_promedio = pd.DataFrame([{
            'Frecuencia Dominante (Hz)': promedio.get('Frecuencia Dominante (Hz)', np.nan),
            'RMS (m/s2)': promedio.get('RMS (m/s2)', np.nan),
            'Amplitud Temblor (cm)': promedio.get('Amplitud Temblor (cm)', np.nan)
        }])
    else:
        df_promedio = pd.DataFrame()

    return df_promedio, df_por_ventana